In [3]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [4]:
data_raw = pd.read_csv('PROJECT/COVID-19 Philippines.csv')
data_clean = data_raw.iloc[:,0:25]
data_clean = data_clean.dropna(how='all')

# Epi Link

In [28]:
data_clean['Epi_Link'].value_counts()

With Exposure to known COVID-19 case                                                                                                                                                                                                               8
With exposure to known COVID-19 case                                                                                                                                                                                                               4
Related to a known COVID-19 case                                                                                                                                                                                                                   3
With exposure history to a known COVID-19 case                                                                                                                                                                                                     2
Contact of PH43     

In [29]:
#PREP
#make everything lowercase so words like Travel and travel are equivalent
data_clean['Epi_Link']=data_clean['Epi_Link'].str.lower()

#make NaNs strings since we are using boolean string comparison and I was getting weird errors
data_clean['Epi_Link']=data_clean['Epi_Link'].fillna("NaN") 

Flag for people who traveled <br>
- Approach: If epi_link contains the word travel, epi_link_traveled == 1

In [30]:
#travel flag
data_clean['Epi_Link_Traveled']=0
for i in range(len(data_clean)):
    if'travel' in data_clean['Epi_Link'][i]:
        data_clean['Epi_Link_Traveled'][i] = 1 

In [31]:
data_clean[data_clean['Epi_Link_Traveled']==1]

,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,Date of Admission,...,Health Status,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Latitude,Longitude,Residence Lat,Residence Long,Epi_Link_Traveled
2,PH00003,F,60.0,Chinese,None,Yes (China),China,traveled to cebu and bohol. was already in chi...,21-Jan-20,22-Jan-2020,...,Recovered,15-Mar-2020,NaN,51 - 60,Manila City,14.615867,120.980991,14.598727,120.981991,1
261,PH00262,M,45.0,Filipino,"Davao City, Davao Del Sur",Came from Manila,For validation,"husband of ph263, both traveled from manila",NaN,NaN,...,Mild,NaN,NaN,41 - 50,"Davao City, Davao Del Sur",14.615867,120.980991,7.188425,125.447792,1
262,PH00263,F,46.0,Filipino,"Davao City, Davao Del Sur",Came from Manila,For validation,"wife of ph262, both traveled from manila",NaN,NaN,...,Mild,NaN,NaN,41 - 50,"Davao City, Davao Del Sur",7.099528,125.619416,7.188425,125.447792,1
441,PH00442,M,28.0,For validation,"Davao City, Davao Del Sur",For validation,For validation,traveled from manila,NaN,NaN,...,For validation,NaN,NaN,21 - 30,"Davao City, Davao Del Sur",7.099528,125.619416,7.188425,125.447792,1
442,PH00443,M,32.0,Filipino,"Davao City, Davao Del Sur",For validation,For validation,traveled from manila,NaN,NaN,...,For validation,NaN,NaN,31 - 40,"Davao City, Davao Del Sur",14.615867,120.980991,7.188425,125.447792,1
443,PH00444,M,50.0,Filipino,Makati City,For validation,For validation,"traveled from manila, palawan, cebu",NaN,NaN,...,For validation,NaN,NaN,41 - 50,Makati City,7.422318,125.828438,14.554498,121.022931,1


Exposure count
- Approach: For instances that aren't about travel or aren't really long (less than 50 characters), split on the comma, colon, or and and count

In [32]:
#epi link exposure
data_clean['Epi_Link_Exposure']=0
for i in range(len(data_clean)):
    if len(data_clean['Epi_Link'][i]) < 50:
        data_clean['Epi_Link'][i]=re.split('; |, | and', data_clean['Epi_Link'][i])                                    
        data_clean['Epi_Link_Exposure'][i] = len(data_clean['Epi_Link'][i])
                                           
    #for now ignore instances that are about travel
    if'travel' in data_clean['Epi_Link'][i]:
        data_clean['Epi_Link_Exposure'][i] = 0 

In [33]:
data_clean.head(10)

,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,Date of Admission,...,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Latitude,Longitude,Residence Lat,Residence Long,Epi_Link_Traveled,Epi_Link_Exposure
0,PH00001,F,38.0,Chinese,None,Yes (China); Wife of PH02,China,[wife of ph2],21-Jan-20,25-Jan-2020,...,15-Mar-2020,NaN,31 - 40,Manila City,14.613480,120.980950,14.598727,120.981991,0,1
1,PH00002,M,44.0,Chinese,None,Yes (China); Husband of PH1,China,[husband of ph1],18-Jan-20,25-Jan-2020,...,1-Feb-2020,Severe Pneumonia,41 - 50,Manila City,14.613480,120.980950,14.598727,120.981991,0,1
2,PH00003,F,60.0,Chinese,None,Yes (China),China,traveled to cebu and bohol. was already in chi...,21-Jan-20,22-Jan-2020,...,15-Mar-2020,NaN,51 - 60,Manila City,14.615867,120.980991,14.598727,120.981991,1,0
3,PH00004,M,48.0,Filipino,Taguig City,Yes (Japan),Japan,[NaN],03-Mar-20,6-Mar-2020,...,19-Mar-2020,"Asymptomatic, Negative Test Result",41 - 50,Taguig City,14.606904,121.020392,14.517274,121.049613,0,1
4,PH00005,M,62.0,Filipino,"Cainta, Rizal",Husband of PH06,None,[husband of ph6],25-Feb-20,5-Mar-2020,...,12-Mar-2020,"Acute Respiratory Distress Syndrome, Severe pn...",61 - 70,"Cainta, Rizal",14.409635,121.037425,14.586506,121.113825,0,1
5,PH00006,F,59.0,Filipino,"Cainta, Rizal",Wife of PH05,None,[wife of ph5],27-Feb-20,5-Mar-2020,...,11-Mar-2020,"Acute Respiratory Distress Syndrome, Severe pn...",51 - 60,"Cainta, Rizal",14.409635,121.037425,14.586506,121.113825,0,1
6,PH00007,M,38.0,"Chinese, Taiwanese",Makati City,No Travel History; Contact of FN2,None,[contact of fn2],03-Mar-20,6-Mar-2020,...,21-Mar-2020,"Asymptomatic, Negative Test Result",31 - 40,Makati City,14.409635,121.037425,14.554498,121.022931,0,1
7,PH00008,M,32.0,Filipino,Pasig City,Yes (Japan),Japan,[NaN],01-Mar-20,8-Mar-2020,...,NaN,NaN,31 - 40,Pasig City,14.555719,121.048688,14.576163,121.084615,0,1
8,PH00009,M,86.0,American,Marikina City,"Yes (USA, Korea); In same household with PH027...","USA, Korea","[in same household with ph27-30, contact of ph31]",01-Mar-20,6-Mar-2020,...,14-Mar-2020,"Multiple Organ Failure, Septic Shock, Acute Re...",81 - 90,Marikina City,14.589620,121.069797,14.650423,121.102348,0,2
9,PH00010,M,57.0,Filipino,Quezon City,Taiwan,Taiwan,case reported that a relative from china (with...,20-Feb-20,7-Mar-2020,...,23-Mar-2020,"Asymptomatic, Negative Test Result",51 - 60,Quezon City,14.613480,120.980950,14.676045,121.043361,0,0


# Dates

In [34]:
#PREP

#convert 3 date columns into type datetime
data_clean['Admission_dt']=pd.to_datetime(data_clean['Date of Admission'], 
                                                  infer_datetime_format=True, errors='coerce')
data_clean['SymptomOnset_dt']=pd.to_datetime(data_clean['Date of Onset of Symptoms'], 
                                                    infer_datetime_format=True, errors='coerce')
data_clean['Final_dt']=pd.to_datetime(data_clean['Date of Final Status (recovered/expired)'], 
                                                  infer_datetime_format=True, errors='coerce')

In [35]:
#diff_admit_symp
data_clean['diff_admit_symp']=data_clean['Admission_dt']-data_clean['SymptomOnset_dt']

In [36]:
data_clean['diff_admit_symp'].value_counts()

7 days     15
4 days     14
2 days     11
5 days     10
1 days     10
9 days      7
8 days      6
3 days      6
6 days      5
0 days      3
14 days     3
15 days     2
10 days     2
11 days     1
13 days     1
16 days     1
-1 days     1
32 days     1
-7 days     1
12 days     1
Name: diff_admit_symp, dtype: int64

Need to look into the negative value cases, may want to drop them if the dates are unreliable

In [37]:
#diff_final_admit
data_clean['diff_final_admit']=data_clean['Final_dt']-data_clean['Admission_dt']

In [38]:
data_clean['diff_final_admit'].value_counts()

13 days    6
12 days    6
6 days     5
8 days     5
7 days     5
10 days    4
9 days     2
17 days    2
19 days    2
-1 days    2
14 days    2
5 days     1
20 days    1
53 days    1
23 days    1
16 days    1
3 days     1
2 days     1
50 days    1
15 days    1
11 days    1
Name: diff_final_admit, dtype: int64

In [39]:
#diff_final_symp
data_clean['diff_final_symp']=data_clean['Final_dt']-data_clean['SymptomOnset_dt']

In [40]:
data_clean['diff_final_symp'].value_counts()

0 days     80
19 days     9
14 days     9
20 days     6
12 days     5
15 days     4
16 days     4
13 days     3
22 days     3
18 days     3
11 days     2
21 days     2
54 days     2
8 days      2
17 days     2
23 days     1
32 days     1
27 days     1
28 days     1
4 days      1
26 days     1
24 days     1
Name: diff_final_symp, dtype: int64

^ this result is weird and we shoud look into it more before using this as a feature

## Reverse Geocoding

In [56]:
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

In [45]:
data_clean['Location']

0          Manila City
1          Manila City
2          Manila City
3          Taguig City
4        Cainta, Rizal
             ...      
3013    For validation
3014    For validation
3015    For validation
3016    For validation
3017    For validation
Name: Location, Length: 3018, dtype: object

In [46]:
data_clean.columns

Index(['Case No.', 'Sex', 'Age', 'Nationality', 'Residence in the Philippines',
       'History of Travel / Exposure', 'Travel History', 'Epi_Link',
       'Date of Onset of Symptoms', 'Date of Admission',
       'Date of Lab Confirmation', 'Date of Announcement to the Public',
       'Admission / Consultation', 'Symptoms', 'Other disease', 'Status',
       'Health Status', 'Date of Final Status (recovered/expired)',
       'Final Diagnosis', 'Age Group', 'Location', 'Latitude', 'Longitude',
       'Residence Lat', 'Residence Long', 'Epi_Link_Traveled',
       'Epi_Link_Exposure', 'Admission_dt', 'SymptomOnset_dt', 'Final_dt',
       'diff_admit_symp', 'diff_final_admit', 'diff_final_symp'],
      dtype='object')

In [53]:
data_clean['geom'] = data_clean['Latitude'].map(str) + ',' + data_clean['Longitude'].map(str)
data_clean['geom'][0]

'14.61348,120.98095'

In [57]:
locator = Nominatim(user_agent='myGeocoder', timeout=60)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

In [59]:
#DO NOT RUN THIS AGAIN TAKES A LONG TIME LOL
tqdm_notebook.pandas()
data_clean['address'] = data_clean['geom'].progress_apply(rgeocode)
data_clean.head()

RateLimiter caught an error, retrying (0/2 tries). Called with (*('14.647836499999999,121.0456111',), **{}).
Traceback (most recent call last):
  File "/Users/laurendarinzo/opt/anaconda3/lib/python3.7/site-packages/geopy/geocoders/base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/Users/laurendarinzo/opt/anaconda3/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/Users/laurendarinzo/opt/anaconda3/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/Users/laurendarinzo/opt/anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/Users/laurendarinzo/opt/anaconda3/lib/python3.7/urllib/request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/Users/laurendarinzo/opt/anaconda3/lib/python3.7/urllib/request.py", line 1320, in do_open
    r = h.getresponse()
  File "/Users/l

,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,Date of Admission,...,Epi_Link_Traveled,Epi_Link_Exposure,Admission_dt,SymptomOnset_dt,Final_dt,diff_admit_symp,diff_final_admit,diff_final_symp,geom,address
0,PH00001,F,38.0,Chinese,None,Yes (China); Wife of PH02,China,[wife of ph2],21-Jan-20,25-Jan-2020,...,0,1,2020-01-25,2020-01-21,2020-03-15,4 days,50 days,54 days,"14.61348,120.98095","(San Lazaro Hospital, Quiricada Street, Barang..."
1,PH00002,M,44.0,Chinese,None,Yes (China); Husband of PH1,China,[husband of ph1],18-Jan-20,25-Jan-2020,...,0,1,2020-01-25,2020-01-18,2020-02-01,7 days,7 days,14 days,"14.61348,120.98095","(San Lazaro Hospital, Quiricada Street, Barang..."
2,PH00003,F,60.0,Chinese,None,Yes (China),China,traveled to cebu and bohol. was already in chi...,21-Jan-20,22-Jan-2020,...,1,0,2020-01-22,2020-01-21,2020-03-15,1 days,53 days,54 days,"14.615867000000001,120.98099099999999","(Francisco Duque Hall, Tayuman Street, Santa C..."
3,PH00004,M,48.0,Filipino,Taguig City,Yes (Japan),Japan,[NaN],03-Mar-20,6-Mar-2020,...,0,1,2020-03-06,2020-03-03,2020-03-19,3 days,13 days,16 days,"14.6069038,121.02039180000001",(University of the East - Ramon Magsaysay Memo...
4,PH00005,M,62.0,Filipino,"Cainta, Rizal",Husband of PH06,None,[husband of ph6],25-Feb-20,5-Mar-2020,...,0,1,2020-03-05,2020-02-25,2020-03-12,9 days,7 days,16 days,"14.409634699999998,121.03742450000001","(Research Institute for Tropical Medicine, Cor..."


In [69]:
data_clean['address'][0]

geopy.location.Location

In [67]:
data_clean.to_csv('data_clean_w_addresses.csv')

## Pulling ZipCode from Geocode Object

In [1]:
!ls

Analysis_Modeling_COVID19.ipynb
COVID_04162020_PHnum.xlsx
CodaLab Instructions
EDA_COVID19_amber-annotated.ipynb
EDA_COVID19_amber.ipynb
Icon?
PROJECT
README.md
data_clean.csv
data_clean_w_addresses.csv
data_covidph
feature_engineering_LD.ipynb
open_refine
openrefine_PHCASES_COVID_04162020_PHnum.csv
openrefine_phcases.rtf


In [61]:
df = pd.read_csv('data_clean_w_addresses.csv')

In [62]:
type(df['address'][1])

str

In [63]:
df['address'][2]

'Francisco Duque Hall, Tayuman Street, Santa Cruz, Manila, Third District, NCR, First District, Metro Manila, 1942, Philippines'

In [64]:
df['address'].str.extract(pat = '([0-9][0-9][0-9][0-9], Philippines)')
df['address2']=df['address'].str.extract(pat = '([0-9][0-9][0-9][0-9], Philippines)')

In [65]:
df['zip']=df['address2'].str.split(",", n = 1, expand = True)[0]

In [66]:
df['zip'].value_counts()

1942    2108
1100      93
1634      85
1112      76
1229      66
        ... 
2600       1
2306       1
2400       1
2016       1
3311       1
Name: zip, Length: 91, dtype: int64

In [67]:
df.head()

,Unnamed: 0,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,...,Admission_dt,SymptomOnset_dt,Final_dt,diff_admit_symp,diff_final_admit,diff_final_symp,geom,address,address2,zip
0,0,PH00001,F,38.0,Chinese,None,Yes (China); Wife of PH02,China,['wife of ph2'],21-Jan-20,...,2020-01-25,2020-01-21,2020-03-15,4 days 00:00:00.000000000,50 days 00:00:00.000000000,54 days 00:00:00.000000000,"14.61348,120.98095","San Lazaro Hospital, Quiricada Street, Baranga...","1003, Philippines",1003
1,1,PH00002,M,44.0,Chinese,None,Yes (China); Husband of PH1,China,['husband of ph1'],18-Jan-20,...,2020-01-25,2020-01-18,2020-02-01,7 days 00:00:00.000000000,7 days 00:00:00.000000000,14 days 00:00:00.000000000,"14.61348,120.98095","San Lazaro Hospital, Quiricada Street, Baranga...","1003, Philippines",1003
2,2,PH00003,F,60.0,Chinese,None,Yes (China),China,traveled to cebu and bohol. was already in chi...,21-Jan-20,...,2020-01-22,2020-01-21,2020-03-15,1 days 00:00:00.000000000,53 days 00:00:00.000000000,54 days 00:00:00.000000000,"14.615867000000001,120.98099099999999","Francisco Duque Hall, Tayuman Street, Santa Cr...","1942, Philippines",1942
3,3,PH00004,M,48.0,Filipino,Taguig City,Yes (Japan),Japan,['NaN'],03-Mar-20,...,2020-03-06,2020-03-03,2020-03-19,3 days 00:00:00.000000000,13 days 00:00:00.000000000,16 days 00:00:00.000000000,"14.6069038,121.02039180000001",University of the East - Ramon Magsaysay Memor...,"1016, Philippines",1016
4,4,PH00005,M,62.0,Filipino,"Cainta, Rizal",Husband of PH06,None,['husband of ph6'],25-Feb-20,...,2020-03-05,2020-02-25,2020-03-12,9 days 00:00:00.000000000,7 days 00:00:00.000000000,16 days 00:00:00.000000000,"14.409634699999998,121.03742450000001","Research Institute for Tropical Medicine, Corp...","1781, Philippines",1781


In [69]:
df=df.drop(columns='address2')

In [70]:
df.columns

Index(['Unnamed: 0', 'Case No.', 'Sex', 'Age', 'Nationality',
       'Residence in the Philippines', 'History of Travel / Exposure',
       'Travel History', 'Epi_Link', 'Date of Onset of Symptoms',
       'Date of Admission', 'Date of Lab Confirmation',
       'Date of Announcement to the Public', 'Admission / Consultation',
       'Symptoms', 'Other disease', 'Status', 'Health Status',
       'Date of Final Status (recovered/expired)', 'Final Diagnosis',
       'Age Group', 'Location', 'Latitude', 'Longitude', 'Residence Lat',
       'Residence Long', 'Epi_Link_Traveled', 'Epi_Link_Exposure',
       'Admission_dt', 'SymptomOnset_dt', 'Final_dt', 'diff_admit_symp',
       'diff_final_admit', 'diff_final_symp', 'geom', 'address', 'zip'],
      dtype='object')

In [71]:
data_clean.to_csv('data_clean_w_zip.csv')